# Using TEA (Taxa Evaluation and Assessment) 


## Quick Start

Create a Misc object, then use the function main().
    
Arguments:
- Input: the name of the ground truth profile file
- Output: the excel file name, a .xlsx of six sheets: True Positives, False Negatives, False Positives, True Negatives, Precision, Recall of each tool
- (optional) input directory of all profiles, including the ground truth; <Default: Directory of Package Manual>
- (optional) the output directory; <Default: Directory of Package Manual>
- (optional) **yes** if you want individual .csv files of each tool's confusion matrix; <Default: **no**>

In [ ]:
from TEA.precall import Misc
Quick = Misc()

In [ ]:
Quick.main("truth.profile", "Test_Tool", "tests", "tests_results")

## How they work

The main purpose of this module is to calculate each Tax ID's confusion matrix and save that and other information into a .csv file.

Each module does a specific step in that process

## Parse

Parse separates the relevant information for each Tax ID (ie. rank, abundance, etc.) into dictionaries. Both functions *parse_data()* and *main()* return a dictionary of dictionaries variable, but *main()* does a little more with it.

The returned variable looks like this:

    {Sample Number : dict}
                   - {Rank : dict}
                           - {Tax ID : Abundance}
There is an alternative parsing format that can be used to get other information about the Tax ID (name and rank), which returns a dictionary of dictionaries variable that looks like this:

    {Sample Number : dict}
                  - {Tax ID : list}
                            - [rank, name]

In [ ]:
from TEA.profile_parser import Parser

myParser = Parser()

### main(self, f, t=0)

This function calls inner functions to create and return a dictionary where sample number is the key and a dictionary of {Rank : {Tax ID : Abundance}} is the value.
    
The variable t is optional and the default is zero. Passing one for t instead of zero tells the program to use the alternative parsing format.

In [ ]:
sample_1 = myParser.main("pred.profile")
sample_2 = myParser.main("pred2.profile")

sample_1_alt = myParser.main("pred.profile", 1)

### print_samples(self, samples, t=0)

This function prints the contents of the dictionary samples in a viewable way.

The variable **t** defaults to zero, printing in the format of the default sample format. If **t=1**, it prints in the alternative format.

In [ ]:
print("~DEFAULT~")
myParser.print_samples(sample_1)
print("~ALTERNATIVE~")
myParser.print_samples(sample_1_alt, 1)

## Comparator

Comparator compares two sample dictionaries to find the common Tax IDs in each sample and to combine the Tax IDs in each sample into a new dictionary.

In [ ]:
from TEA.comparator import Comparator

myComparer = Comparator()

### main(self, files, t=0)

The *main()* function takes in a string (file names) of two profile names (with the '.profile' part) separated by a space. If the profile files are not in the same directory as the file running this code, add the directory to the file names. Make sure there are no spaces in the directory path.

It also accepts an optional argument **t** that is used when the function calls on *Parser.main()*.
    
This function returns two dictionaries where sample number is the key and a set is the value. The sets for the two dictionaries contain all the Tax IDs from both samples, excluding repeats, and the Tax IDs both samples had in common, respectively.

In [ ]:
common_1_2, combined_1_2 = myComparer.main("tests\\truth.profile tests\\pred.profile")
print(common_1_2)
print(combined_1_2)

### save_tax_ID(self, samples)

This function iterates over a samples dictionary to create and return a dictionary where sample number is the key and a set of Tax IDs from that sample is the value.

In [ ]:
sample_tax_ID_1 = myComparer.save_tax_ID(sample_1)
sample_tax_ID_2 = myComparer.save_tax_ID(sample_2)
print(sample_tax_ID_1)
print(sample_tax_ID_2)

### common_tax_ID(self, tax_id1, tax_id_2)

This function creates and returns a dictionary where sample number is the key and a set containing the common Tax IDs between two sample files is the value. It iterates over a samples dictionary to save each set under a sample number.

In [ ]:
common_dict_1_2 = myComparer.common_tax_ID(sample_tax_ID_2, sample_tax_ID_1)
print(common_dict_1_2)

### combine_tax_ID(self, tax_id_1, tax_id_2)

This function creates and returns a dictionary where sample number is the key and a set containing the Tax IDs from both sample dictionaries is the value.

In [ ]:
combined_dict_1_2 = myComparer.combine_tax_ID(sample_tax_ID_2, sample_tax_ID_1)
print(combined_dict_1_2)

## Confusion

Confusion uses both Comparator and Parser to create a confusion matrix for every Tax ID in each sample.

In [ ]:
from TEA.confusion_matrix import Confusion

### \__init__(self, tru, fn)

The constructor for Confusion objects is a little different since it takes two arguments, one for the name of the ground truth file and one for the name of the predicted file (including the '.profile' part).

If the files are not in the same directory as the file running this code, add the directory path to the names

In [ ]:
Confu = Confusion("tests\\truth.profile", "tests\\pred.profile")

### get_file_name(self) and get_truth(self)

These functions return a string containing the name of the predicted and truth files respectively

In [ ]:
print("Truth:", Confu.get_truth(), '\nPredicted:', Confu.get_file_name())

### set_file_name(self, tru) and set_truth(self, fn)

These two functions allow you to change the ground truth or predicted file whenever you need to. It also makes it so that you only need one Confusion object for multiple predicted files or even multiple truth files.

In [ ]:
Confu.set_file_name("tests\\pred2.profile")
Confu.set_truth("tests\\pred.profile")
print("Truth:", Confu.get_truth(), '\nPredicted:', Confu.get_file_name())

### main(self, csv="yes", t=0)

This function uses *Comparer* and *Parser* and internal functions to create a confusion matrix for the predicted file, then save that data as a .csv file. The automatic name for that .csv file is the truth and predicted file names joined by two hyphens (ie. Truth = 'truth.profile', Predicted = 'pred.profile', .csv file name = 'truth--pred.csv').
    
It also returns the confusion matrix it created

In [ ]:
t_p_matrix = Confu.main()

### check_matrix_error(self, matrix)

This a supplementary function that checks to make sure the sum of the numbers in the confusion matrix equal the total number of samples in the ground truth file for each Tax ID. It then returns a list of the Tax IDs with confusion matrices that are over and under that number respectively.

In [ ]:
Confu.check_matrix_error(t_p_matrix)

## Misc

*Misc* is mainly for finding the confusion matrices of multiple profiles at once and creating an .xlsx file to display the confusion matrix values for each predicted file and the ground truth on separate sheets. It also uses the Precall class to calculate and add a sheet for precision and recall.

In [ ]:
from TEA.precall import Misc

myMisc = Misc()

### set_truth(self, truth)

This function sets the file name of the ground truth

In [ ]:
myMisc.set_truth("new_truth.profile")
print(myMisc.get_truth())

### set_input_path(self, ip)

This function sets the directory path for the input files (ground truth and predicted profile files).

In [ ]:
myMisc.set_input_path('new_input_path')
print(myMisc.get_input_path())

### set_output_path(self, op)

This function sets the directory path for the output files (i.e. the .xlsx file and dendrograms)

In [ ]:
myMisc.set_output_path('new_outputs')
print(myMisc.get_output_path())

### set_output_name(self, on)

This function sets the name of the .xlsx output file.

In [ ]:
myMisc.set_output_name('A_Different_Name')
print(myMisc.get_output_name())

### main(self, gnd_truth, excel_name="TaxaPerformanceMetrics_byTool", gen_dir='"", file_path="",  csv="no")

This function takes the name of the ground truth file, the name of the output .xlsx file, the directory of the profile files, the directory for the output file, and a string to  determine whether or not to also create .csv files for each tool's confusion matrix.
- The name of the ground truth file is a string.
- The name of the output .xlsx file can be anything you choose. The default is "TaxaPerformanceMetrics_byTool".
- The directory of the profile files needs to include all the profiles being evaluated plus the ground truth file. They should all be in the CAMI format and have the extension '.profile'. The default is the package manual directory.
- The directory for the .xlsx output file can be anything you choose. The default is the package manual directory.
- The .csv files contain the confusion matrix for each Tax ID. One file would be made for each tool. If you want the progam to output them, pass in **yes**, otherwise leave blank since the default is **no**. The directory for these files will be the same as the .xlsx output file's.
        
After saving the output excel file, dendrograms based on the bray curtis distance are created and saved as .png files. These are created for each taxa level for True Positives, False Negatives, False Positives, and True Negatives to compare the tools to each other. There is also a dendrogram for each confusion matrix metric that includes all taxa levels and uses traceback to include the Tax IDs that weren't directly reported but were detected. The .png files are also saved in the same directory as the output excel file.

If you want the ground truth file to be evalutated with the predicted files, make a copy under a different name. In the 'tests' folder of TEA, *gnd.profile* is a copy of the ground truth file, *truth.profile*.

In [ ]:
myMisc.main("truth.profile", "Test_Tool", "tests", "tests_results")

### get_top_taxid(self, x, metric="tp", difficulty="easy", truth="yes")

This function finds and creates a .xlsx file of the x number of taxa with the top difficulty passed based on the metric chosen.

*Before running this, the main() function should be run or these values should be set: the ground truth file name, output file directory, and output file name.*

- The ground truth file name should be the same as the one used for the main() .xlsx output file.
- The output file directory is the same as the one used for the main() .xlsx output file.
- The output file name is the main() .xlsx output file name.
    
__*x*__ is the number of Tax IDs to include
    
*__metric__* is the way the top taxa will be calculated. It can be True Positives (*tp*), False Negatives (*fn*), False Positves (*fp*), True Negatives (*tn*), or the average of Precision and Recall (*precall*). *TP* is the default.
    
__*Difficulty*__ is what kind of Tax ID you're looking for. *Easy*, *Hard*, and *Nan* can be passed to get the top easiest, hardest, or not-a-number Tax IDs (Note: only the *precall* option really has 'nan' values). The default is *easy*.
    
The __*truth*__ argument tells the program whether or not you want to include Tax IDs only from the ground truth. If *yes* is passed, only Tax IDs from the ground truth will be included in the top taxa. If *no* is passed, all taxa between all predicted files and the ground truth will be included. *Yes* is the default.
    
*None of these arguments are case-sensitive.*
    
The top taxa will be saved into a .xlsx file in whichever output directory was last passed. 

In [ ]:
myMisc.get_top_taxid(3, 'tp', 'hard', 'yes') # Gets the top three hardest Taxa based on True Positive values

### create_heat_map(self, file_name)

This function creates a heat map with a dendrogram on top using a list of Tax IDs from the file passed.

*Before running this, the main() function should be run or the output file path and name should be set with set_output_path() and set_output_name().*
    
**file_name** is the name of the spreadsheet with the list of Tax IDs, which should be grouped into a column labeled 'Tax ID'.
    
After the heat map is created, it will be saved as a .png file iin whichever output directory was last passed.

In [ ]:
myMisc.create_heat_map('Top_Hard-TP_taxid.xlsx') # creates a heat map based on the list of Tax IDs previously calculated